# Demonstration
End-to-end demonstration.<br>
Ref. https://docs.google.com/document/d/1opi0xGTaMZC0TA2CF2VF1DrWHXeE6M-89ricwlsXtXg/edit?usp=sharing

In [ ]:
import utils.DemoClient as client
import jwt
import json
import boto3
from awscli.customizations.s3.utils import split_s3_bucket_key

from owslib.csw import CatalogueServiceWeb
from owslib.fes import And, Or, PropertyIsEqualTo, PropertyIsGreaterThanOrEqualTo, PropertyIsLessThanOrEqualTo, PropertyIsLike, BBox, SortBy, SortProperty
from owslib.wms import WebMapService
from geolinks import sniff_link
import folium
import requests
from bs4 import BeautifulSoup
import pyops

## Demo Client Setup
We now instantiate a client to interact with the platform.<br>
The client dynamically registers with the Authorization Server to take part in UMA (User Managed Access) flows through which authorization is obtained for scoped access resources on behalf of the user.

In [ ]:
#-------------------------------------------------------------------------------
# Initialise client
#-------------------------------------------------------------------------------
base_domain = "185.52.193.87.nip.io"
demo = client.DemoClient(f"https://test.{base_domain}")
demo.register_client()
demo.save_state()

### Authenticated User (Eric)
We require an authenticated user to interact with some of the platform endpoints - in particular for eoepca v0.3 the ADES.<br>
User authenticates and the client receives an ID Token (JWT) that represents the user.<br>
For convenience within the Jupyter notebook we use a username/password authentication - but the primary mechanism is to rely upon external identity provision.<br>
Currently, login with GitHub is supported.

In [ ]:
#-------------------------------------------------------------------------------
# Authenticate as user 'eric' and get ID Token
#-------------------------------------------------------------------------------
USER_NAME="eric"
USER_PASSWORD="defaultPWD"
user_id_token = demo.get_id_token(USER_NAME, USER_PASSWORD)

## Application Discovery
Alice uses the Processor Development Environment (PDE) to develop, test and package an application.
Alice's published outputs are:
* Docker image published to DockerHub
* Application Package (CWL) published to Resource Catalogue (TBD) and/or GitHub - accessible by href

In [ ]:
app_id = "nhi"
app_version = "0_0_3"
application_name = f"{app_id}-{app_version}"

The following demonstrates how to discover the Application Package from Alice's workspace through the Resource Catalogue API

In [ ]:
# TODO - Alice's workspace not working
#alice_workspace_endpoint = f'https://resource-catalogue.rm-user-alice.{base_domain}'
alice_workspace_endpoint = f'https://resource-catalogue.user.{base_domain}'
csw = CatalogueServiceWeb(alice_workspace_endpoint,timeout=30)
csw.getrecords2(maxrecords=10)
print("Alice's workspace records:")
for rec in csw.records:
    print(f'identifier: {csw.records[rec].identifier}\ntype: {csw.records[rec].type}\ntitle: {csw.records[rec].title}\n')
print("URL for Application s-expression:")
csw.records[app_id].references

In [ ]:
# Extract the URL for the Application Package
# TODO [Angelos] - need to extract http URL for AP
application_package_url = "TBD"
print(f"Application Package URL = {application_package_url}")

## Data Discovery

Eric discovers data using platform services (Resource Catalogue / DAS) and QGIS
* CSW and OpenSearch endpoints
* Data visualisation

### Resource Catalogue CSW and OpenSearch

In this part of the demo, Eric uses the system level resource catalogue endpoint to discover data collections and datasets.  The system level resource catalogue enables discovery of data available to all users.

[OWSLib](https://geopython.github.io/OWSLib) is a Python package for client programming with Open Geospatial Consortium (OGC) web service (hence OWS) interface standards, and their related content models. In this demo we will work with the CSW, WMS and WCS interfaces.

The `owslib.csw` class of OWSLib is instantiated and service metadata are shown.

In [ ]:
system_catalogue_endpoint = f'https://resource-catalogue.{base_domain}'
csw = CatalogueServiceWeb(system_catalogue_endpoint, timeout=30)

Service metadata shown here include supported operations and queryables

In [ ]:
[op.name for op in csw.operations]

In [ ]:
csw.get_operation_by_name('GetRecords').constraints

Eric can make a GetRecords request to get all records of the catalogue, with a page size of 10.

In [ ]:
csw.getrecords2(maxrecords=10)
csw.results

Eric wishes to discover data with specific search criteria. Here we demonstrate how to create spatial (`bbox`), temporal (`time`), and attribute (`apiso:CloudCover`) filters combined with logical operators like And/Or. An OGC Filter is created and sent to the Resource Catalogue:

In [ ]:
bbox_query = BBox([37, 13.9, 37.9, 15.1])
begin = PropertyIsLessThanOrEqualTo(propertyname='apiso:TempExtent_begin', literal='2021-04-02 00:00')
end = PropertyIsLessThanOrEqualTo(propertyname='apiso:TempExtent_end', literal='2021-04-02 23:59')
cloud = PropertyIsLessThanOrEqualTo(propertyname='apiso:CloudCover', literal='20')
filter_list = [
    And(
        [
            bbox_query,  # bounding box
            begin, end,  # start and end date
            cloud        # cloud
        ]
    )
]
csw.getrecords2(constraints=filter_list, outputschema='http://www.isotc211.org/2005/gmd')
csw.results

In [ ]:
for rec in csw.records:
    print(f'identifier: {csw.records[rec].identifier}\ntype: {csw.records[rec].identification.identtype}\ntitle: {csw.records[rec].identification.title}\n')

Eric selects one record using the identifier and discovers the available links:

In [ ]:
selected_record = list(csw.records)[0] # TODO [Angelos] - check which is the best scene
csw.getrecordbyid(id=[selected_record])
rec = csw.records[selected_record]
rec.title

In [ ]:
rec.references

Eric can also see the record footprint on a map, using the [Folium](https://github.com/python-visualization/folium) Python library:

In [ ]:
m = folium.Map(location=[38, 20], zoom_start=6, tiles='OpenStreetMap')
folium.Rectangle(bounds=[[float(rec.bbox.miny), float(rec.bbox.minx)], [float(rec.bbox.maxy), float(rec.bbox.maxx)]]).add_to(m)
m

The same record can also be retrieved from the Resource Catalogue using the OpenSearch API and the dataset identifier:

In [ ]:
url=f'{system_catalogue_endpoint}/?mode=opensearch&service=CSW&version=3.0.0&request=GetRecords&elementsetname=full&resulttype=results&typenames=csw:Record&recordids={selected_record}'
S = requests.Session()
R = S.get(url=url)
bs = BeautifulSoup(R.text, 'xml')
print(bs.prettify())

Eric can also use a generic OpenSearch client to query the Resource Catalogue, including parameters in the search, like the parent identifier for collection level search:

In [ ]:
opensearch_endpoint=f'{system_catalogue_endpoint}/?service=CSW&version=3.0.0&request=GetCapabilities&mode=opensearch'
client = pyops.Client(description_xml_url=opensearch_endpoint)
results = client.search(params={"{eo:parentIdentifier?}": {"value": "S2MSI2A"}})
len(results)

### Data Access Services - WMS, WCS

Eric has already discovered the dataset to use by using the OWSLib CSW client.

Using the [geolinks](https://github.com/geopython/geolinks) Python library, Eric can filter the links that are of a specific type (here WMS and WCS links to be used for visualization):

In [ ]:
csw = CatalogueServiceWeb(system_catalogue_endpoint, timeout=30)
scene_id = selected_record
csw.getrecordbyid(id=[scene_id])
links = csw.records[scene_id].references
for link in links:
    scheme = link['scheme']
    if 'WMS' in scheme:
        wms_endpoint=link['url']
        print(link['url'])

After discovering the dataset, Eric can identify the WMS link and use the OWSLib WMS client to visualize the dataset

In [ ]:
wms = WebMapService(wms_endpoint, version='1.3.0')
list(wms.contents)

In [ ]:
wms[scene_id].title

Eric can now visualize the WMS GetMap request from matplotlib:

In [ ]:
%matplotlib inline
import os, sys
import matplotlib.image as mpimg
import matplotlib.pyplot as plt

def get_map(wms, layer_name, bbox, filename):
    wms.getOperationByName('GetMap').formatOptions
    img = wms.getmap(layers=[layer_name],
                     size=(600,300),
                     srs='EPSG:4326',
                     bbox=bbox,
                     format='image/png',
                     transparent=True)

    with open(filename,'wb') as tmpfile:
        tmpfile.write(img.read())

In [ ]:
get_map(wms,f'{selected_record}__TRUE_COLOR',(13.8,36.9,15.1,37.9), 'rgb.png')
image1 = mpimg.imread('rgb.png')
fig = plt.figure(figsize=(12,7))
img1 = plt.imshow(image1, extent=[13.8,15.1,36.9,37.9], aspect='auto')
plt.show()

## Processing - ADES
The ADES provides WPS 1.0/2.0 and OGC API - Processes interfaces - with extensions for deploying and undeploying processes.

The ADES provides user-specific endpoints, using a URL path prefix.

In [ ]:
# ADES URLs
ades_base_url = f"http://ades.test.{base_domain}"
ades_wps_url = f"{ades_base_url}/{USER_NAME}/zoo"
print("ADES WPS endpoint:", ades_wps_url)

ades_proc_url = f"{ades_base_url}/{USER_NAME}/wps3"
print("ADES API Processes endpoint:", ades_proc_url)

# Init
ades_access_token = None

### ADES: List Processes
**GET {service_url}/processes**<br>
Provides a list of all processes 

In [ ]:
# API Processes - List Processes
response, ades_access_token, process_ids = demo.proc_list_processes(ades_proc_url, id_token=user_id_token, access_token=ades_access_token)
print("Processes:", process_ids)
# demo.response_summary(response)

### ADES: Deploy Application
**POST {service_url}/processes**<br>
Deploy application to the ADES.<br>
Request body is json in the same format as defined for an Execute request (ref. API Processes), in which the input is the Application Package describing the application.<br>
The Application Package is a CWL Workflow that is typically provided as an href with content type **_application/atom+xml_** or **_application/cwl_**.

In [ ]:
# TODO - dynamic insertion of AP URL into deploy POST body
# something like...
# sed -e 's/APPLICATION_PACKAGE_URL/<ap-url>/' demoroot/data/app-deploy-body-template.json

In [ ]:
# API Processes - Deploy Application
response, ades_access_token = demo.proc_deploy_application(ades_proc_url, "../data/app-deploy-body-cwl.json", id_token=user_id_token, access_token=ades_access_token)

**_Dynamic Resource Protection_**<br>
The application deployment creates new endpoints within the ADES that need to be protected - e.g. those for _Details_ **(processes/{application_name})** and _Execute_ **(processes/{application_name}/jobs)**.<br>
During deployment, the ADES interfaces with its PEP (Policy Enforcement Point) to register these endpoints as protected under the ownership of the calling user.

In [ ]:
# Check deploy - list processes again
response, ades_access_token, process_ids = demo.proc_list_processes(ades_proc_url, id_token=user_id_token, access_token=ades_access_token)
print("Processes:", process_ids)

### ADES: Get Application Details
**GET {service_url}/processes/{application_name}**<br>
Provides details of the deployed application<br>
The response includes the API Processes json application description.

In [ ]:
# API Processes - Get Application Details
response, ades_access_token = demo.proc_get_app_details(ades_proc_url, application_name, id_token=user_id_token, access_token=ades_access_token)
demo.response_summary(response)

### ADES: Execute Application
**POST {service_url}/processes/{application_name}/jobs**<br>
Request body is json as defined by API Processes to define the inputs and outputs, consistent with the CWL Workflow application package.<br>
The response returns **201 CREATED** to indicate that the job has been successfully initiated.<br>
The response **Location header** provides the path (/processes/{application_name}/jobs/{job_id}) to follow the job status.

In [ ]:
# TODO - dynamic insertion of discovered input URL into execute POST body

In [ ]:
# API Processes - Execute Application
response, ades_access_token, job_location_path = demo.proc_execute_application(ades_proc_url, application_name, "../data/app-execute-body.json", id_token=user_id_token, access_token=ades_access_token)

In [ ]:
# Retrieve job id from the Location header of the response
jobid = response.headers["Location"].rsplit('/', 1)[-1]
print(jobid)

### ADES: Job Status
**GET {service_url}/watchjob/processes/{application_name}/jobs/{job_id}**<br>
Check the status of a previously submitted job - using the URL returned in the Location header of the execute request.<br>
The response body json provides a status string (success/running/failed) and a % progress.<br>
In the case of a failure then a descriptive message is provided.

In [ ]:
# API Processes - Job Status
response, ades_access_token, status = demo.proc_get_job_status(ades_base_url, job_location_path, id_token=user_id_token, access_token=ades_access_token)
demo.response_summary(response)

**Polling for Job Completion**<br>
The job execution is asynchronous.<br>
The Job Status endpoint is polled until the job completes.

In [ ]:
# TODO - fix problem with occasional 504 responses from the ades
#   - either fix in the ades
#   - or handle in the demo client
# API Processes - Job Status (keep polling for completion)
response, ades_access_token, status = demo.proc_poll_job_completion(ades_base_url, job_location_path, interval=10, id_token=user_id_token, access_token=ades_access_token)

In [ ]:
# Inspect response
demo.response_summary(response)

### ADES Stage-out
At the successful completion of processing, the ADES stages out the results to platform storage (under the direction of the user's Workspace component), and registers the data in the User Workspace for access and further exploitation.<br>
![ADES processing results stage-out](https://raw.githubusercontent.com/EOEPCA/eoepca/develop/technical/workspace/seq-ades-processing-results-stage-out.png)

### ADES: Job Result
**GET {service_url}/watchjob/processes/{application_name}/jobs/{job_id}/result**<br>
Returns details of the outputs for a successful job execution.<br>
The response body provides json data that includes the reference to the STAC file that indexes the processing outputs.

In [ ]:
# API Processes - Job Result
response, ades_access_token, stacCatalogUri = demo.proc_get_job_result(ades_base_url, job_location_path, id_token=user_id_token, access_token=ades_access_token)
demo.response_summary(response)
print("stacCatalogUri:", stacCatalogUri)

**Inspecting the processing results**

Retrieve access information to the user workspace S3 bucket (bucket name, access key and secret access key)

In [ ]:
# Init
workspace_url = "https://workspace-api." + base_domain
workspace_access_token = None

# Workspace - Get Details
workspace_name = "rm-user-" + USER_NAME.lower()
response, workspace_access_token = demo.workspace_get_details(workspace_url, workspace_name, id_token=user_id_token, access_token=workspace_access_token)
workspace_details = response.json()

# Bucket details
bucket_name = workspace_details["storage"]["credentials"]["bucketname"]
s3_access = workspace_details["storage"]["credentials"]["access"]
s3_secret = workspace_details["storage"]["credentials"]["secret"]

Print the content of the result stac catalog file

In [ ]:
# Quick hack S3 access
import boto3

# Init S3 session for Creodias
S3_ENDPOINT = "https://cf2.cloudferro.com:8080"
session = boto3.session.Session()
s3resource = session.resource('s3', aws_access_key_id=s3_access, aws_secret_access_key=s3_secret, endpoint_url=S3_ENDPOINT)

# Retrieve S3 key path of the catalog.json
stacCatalogUriS3Key = "/".join(stacCatalogUri.rsplit('/', 2)[-2:])
print(f"Content of {stacCatalogUriS3Key} :\n")

# Printing catalog.json
obj = s3resource.Object(bucket_name, stacCatalogUriS3Key)
catalogJson = json.loads(obj.get()['Body'].read().decode('utf-8'))
print(json.dumps(catalogJson, indent=4, sort_keys=True))

List all the files contained in the result folder

In [ ]:
# List result contents
bucket = s3resource.Bucket(bucket_name)
for obj in bucket.objects.filter(Prefix=f"wf-{jobid}/"):
    print(obj.key)

### ADES: List Jobs
**GET {service_url}/processes/{application_name}/jobs**<br>
Provides a list of all jobs for the named processes 

In [ ]:
# API Processes - List Jobs
response, ades_access_token, job_ids = demo.proc_list_jobs(ades_proc_url, application_name, id_token=user_id_token, access_token=ades_access_token)
print("Jobs:", job_ids)

### ADES: Undeploy Application
**DELETE {service_url}/processes/{application_name}**<br>
Undeploy application from the ADES

In [ ]:
# API Processes - Undeploy Application
response, ades_access_token = demo.proc_undeploy_application(ades_proc_url, application_name, id_token=user_id_token, access_token=ades_access_token)

In [ ]:
# Check undeploy - list processes
response, ades_access_token, process_ids = demo.proc_list_processes(ades_proc_url, id_token=user_id_token, access_token=ades_access_token)
print("Processes:", process_ids)

## Workspace

In [ ]:
# Init
workspace_url = "https://workspace-api." + base_domain
workspace_access_token = None
print("workspace_url:", workspace_url)

### Workspace: Get Details

In [ ]:
# Workspace - Get Details
workspace_name = "rm-user-" + USER_NAME.lower()
response, workspace_access_token = demo.workspace_get_details(workspace_url, workspace_name, id_token=user_id_token, access_token=workspace_access_token)
workspace_details = response.json()
demo.response_summary(response)

### Inspect S3 Bucket

In [ ]:
# Bucket details
bucket_name = workspace_details["storage"]["credentials"]["bucketname"]
s3_access = workspace_details["storage"]["credentials"]["access"]
s3_secret = workspace_details["storage"]["credentials"]["secret"]
# Init S3 session for Creodias
S3_ENDPOINT = "https://cf2.cloudferro.com:8080"
session = boto3.session.Session()
s3resource = session.resource('s3', aws_access_key_id=s3_access, aws_secret_access_key=s3_secret, endpoint_url=S3_ENDPOINT)
bucket = s3resource.Bucket(bucket_name)

In [ ]:
#stacCatalogUri = 's3://1a85398587cc4b5795dab4ab565dad34:rm-user-eric/wf-d8be29d6-a986-11eb-beca-7258940a7c04/catalog.json'

# Unpick s3 URL
_, stacCatalogKeyName = split_s3_bucket_key(stacCatalogUri)
print("stacCatalogKeyName:", stacCatalogKeyName)

# Read the STAC file
stacFileObject = list(bucket.objects.filter(Prefix=stacCatalogKeyName))[0]
stacFileJson = json.loads(stacFileObject.get()['Body'].read())
print(json.dumps(stacFileJson, indent=2))

## Results Consumption

* Results discovery - User's resource catalogue
* Results visualisation:
  * Data access services
  * QGIS

In this part of the demo, Eric will use the workspace resource catalogue endpoint to discover processing outputs and applications.  The workspace catalogue is local to the user, which also has federated access to the system catalogue.

In [ ]:
workspace_endpoint = f'https://resource-catalogue.rm-user-{USER_NAME}.{base_domain}'
csw = CatalogueServiceWeb(workspace_endpoint, timeout=30)
csw.getrecords2(maxrecords=10)
csw.results

In [ ]:
for rec in csw.records:
    print(f'identifier: {csw.records[rec].identifier}\ntype: {csw.records[rec].type}\ntitle: {csw.records[rec].title}\n')

In [ ]:
# TODO - need to select the correct scene-id output from the processing
scene_id='S2B_MSIL1C_20210402T095029_N0300_R079_T33SVB_20210402T121737'
csw.getrecordbyid(id=[scene_id])
links = csw.records[scene_id].references
csw.records[scene_id].references

for link in links:
    scheme = link['scheme']
    if 'WMS' in scheme:
        wms_endpoint=link['url']
        print(link['url'])

In [ ]:
wms = WebMapService(wms_endpoint, version='1.3.0')
list(wms.contents)

In [ ]:
wms[scene_id].boundingBoxWGS84

In [ ]:
getMap(wms,'INDEX_S2A_MSIL2A_20191216T004701_N0213_R102_T53HPA_20191216T024808',(13.8,36.9,15.1,37.9), 'rgb.png')
image1 = mpimg.imread('rgb.png')
fig = plt.figure(figsize=(12,7))
img1 = plt.imshow(image1,extent=[13.8,15.1,36.9,37.9],aspect='auto')
plt.show()

## Policy Enforcement and Resource Sharing
Owners of resources can choose to share these with other users by updating the access policies

### Deploy Application
The demo user redeploys de application

In [ ]:
# API Processes - Deploy Application
response, ades_access_token = demo.proc_deploy_application(ades_proc_url, "../data/app-deploy-body-atom.json", id_token=user_id_token, access_token=ades_access_token)

In [ ]:
# Check deploy - list processes again
response, process_ids, ades_access_token = demo.proc_list_processes(ades_proc_url, id_token=user_id_token, access_token=ades_access_token)
print("Processes:", process_ids)

### UserB Authenticates in the Platform
User authenticates and the client receives an ID Token (JWT) that represents the user.

In [ ]:
#-------------------------------------------------------------------------------
# Authenticate as UserB and get ID Token
#-------------------------------------------------------------------------------
USERB_NAME="demoB"
USERB_PASSWORD="defaultPWD"
userb_id_token = demo.get_id_token(USERB_NAME, USERB_PASSWORD)

### UserB Attempts to Execute Demo User Apps
All resources are registered with ownership policies by default

In [ ]:
#-------------------------------------------------------------------------------
# Unauthorized Execution 
# Expected Value: 401
#-------------------------------------------------------------------------------
response, ades_access_token, job_location_path = demo.proc_execute_application(ades_proc_url, application_name, "../data/app-execute-body.json", id_token=None, access_token=None)

### Owner Updates Access Policy
Grants access to UserB to execute the deployed app

In [ ]:
#-------------------------------------------------------------------------------
# Get Ownership Id
#-------------------------------------------------------------------------------
owB = demo.get_ownership_id(userb_id_token)
owA = demo.get_ownership_id(user_id_token)

#-------------------------------------------------------------------------------
# Finds the policy by retrieving a resource_id
# Update policy with allowing access to UserB
#-------------------------------------------------------------------------------
ades_res="http://ades.resources.{base_domain}"
res_id = demo.get_resource_by_name(ades_res, "s-expression-0_0_2", user_id_token)
data={'name':'Updated Execution','description':'modified','config':{'resource_id':'res_id','action':'view','rules':[{'OR':[{'EQUAL':{'id':'owA'}},{'EQUAL':{'id':'owB'}}]}]},'scopes':['protected_access']}
pdp_url= "http://test.{base_domain}/pdp"
resp, text= demo.update_policy(pdp_url, data, res_id, user_id_token)

### UserB Executes Application Succesfully
Shared resources can now be executed

In [ ]:
#-------------------------------------------------------------------------------
# User B Execute Application Succesfully
# Expected Value: 201
#-------------------------------------------------------------------------------
response, ades_access_token, job_location_path = demo.proc_execute_application(ades_proc_url, application_name, "../data/app-execute-body.json", id_token=userb_id_token, access_token=None)